In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from textwrap import dedent
from uuid import uuid4

In [ ]:
# SSF 몰

# 1. Selenium 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://www.lfmall.co.kr/app/product/HSTS5B206B2?has_init=Y"
# url = "https://www.lfmall.co.kr/app/exhibition/menu/502"
url = "https://www.lfmall.co.kr/app/product/HZTS5B894BK"
url = "https://www.ssfshop.com/ranking?rankSect=CLICK_RANK&ctgryFlterCd=CTGRY_FEMALE&preferAgeCd=ALL&brndShopId=&brandShopNo=&dspCtgryNo=&otltYn=&cnncCtgryNo=&lagSpcltyYn=&utag="

driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

# 2. 스크롤 내리기
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 스크롤 맨 아래로 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 로딩 대기
    time.sleep(2)

    # 현재 페이지 높이 확인
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 더 이상 스크롤이 내려가지 않으면 종료
    if new_height == last_height:
        break
    last_height = new_height

# 3. 상품 리스트 만들기
product_list = driver.find_elements(By.CSS_SELECTOR, "#clickNowForm > section > div:nth-child(2) > div.list_goods > ul > li > a")
print(len(product_list))

# 4. 각 url 따기
urls = []
for product in product_list:
    url = product.get_attribute("href")
    urls.append(url)
print(len(urls))

driver.quit()

In [ ]:
# url 저장
import os
os.makedirs("../data/raw_docs", exist_ok=True)
with open("../data/raw_docs/ssf_mall_urls.txt", "w") as f:
    f.write("\n".join(urls))

In [ ]:
# url 불러오기
def load_urls(file_path):
    with open(file_path, "r") as f:
        urls = f.read().splitlines()
    return urls
urls = load_urls("../data/raw_docs/ssf_mall_urls.txt")

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import time

# 5. LLM 모델 준비
model = ChatOpenAI(model="gpt-4.1", temperature=0)

# 6. Prompt Template (중괄호 이스케이프)
prompt_template = ChatPromptTemplate.from_template(dedent("""
다음은 상품 상세 페이지의 HTML 일부 소스입니다.

이 HTML에는 상품에 대한 다양한 정보가 포함되어 있습니다.  
다음 조건에 맞게 JSON 형식으로 변환해주세요:

1. 상품과 관련된 정보는 가능한 한 모두 담아주세요.  
2. HTML 내 존재하는 정보는 최대한 구체적으로 추출해 주세요.  
3. 아래와 같은 JSON 구조를 따르세요. 단, 절대로, '```json', 'json', ' \"\"\" ', '**설명**' 등과 같이 불필요한 값은 채워넣지 않습니다.
4. 만약 특정 항목이 HTML에 없다면 해당 항목은 생략하지 말고 빈 문자열("")로 채워 주세요.  
5. id 항목은 "{product_id}" 로 채워주세요. 쌍따옴표도 포함입니다. id, name, category, description 항목은 필수로 채워야합니다.
6. specifications과 features 항목은 표, 리스트, 상세 스펙 등이 있을 경우나 제품에 대한 서술이 있을 경우 포함해주세요.
7. 배송, 카드사 혜택, 포인트 적립 등의 공통 정보는 제외하고 상품에 직접적으로 관련된 정보만 포함하세요.
8. features 항목은 제품의 특징을 포함합니다. 예를 들어 "깔끔한 디자인", "고급스러운 소재", "가벼운 코튼 소재", "봄부터 여름까지 착용 가능한 제품" 등과 같습니다.
9. category 항목은 상품의 카테고리 또는 분류명을 포함합니다. 예를 들어 "여성 의류 가디건", "남성 하의 팬츠", "액세서리 귀걸이" 등입니다.
10. description 항목은 상세 설명을 포함합니다. 길어도 좋으며, 제품의 특징을 최대한 포함하고자 합니다.
11. 단, 전체 내용은 20만 토큰 이내로 유지해야 합니다.

JSON 출력 예시 형태:

{{
  "id": {product_id}
  "name": "상품명",
  "category": "카테고리 또는 분류명",
  "description": "상세 설명. 길어도 좋음.",
  "price": "가격 정보 (할인 전/후 가격 포함 가능)",
  "specifications": [
    "스펙 1",
    "스펙 2",
    ...
  ],
  "features": [
    "특징 1",
    "특징 2",
    ...
  ],
  "keywords": [
    "키워드 1",
    "키워드 2",
    ...
  ]
}}

다음은 HTML 소스입니다:
{input_data}

"""))

url_test = urls[:50]
# 7. 각 상품별로 들어가서 html 가져오기
for idx, url in enumerate(urls[52:], start=1):
    e = time()
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        target_1 = soup.select_one("#content > section > div.gods-summary > div.godsInfo-area")
        target_2 = soup.select_one("#godsTabView > div.gods-detail-desc.sticky-start > div")
        

        # 둘 중 없는 경우 대비
        target_1_text = str(target_1) if target_1 else ""
        target_2_text = str(target_2) if target_2 else ""

        # 두 영역 합치기
        extracted_html = target_1_text + "\n\n" + target_2_text

        # 체인 실행
        product_id = uuid4()
        chain = prompt_template | model
        result = chain.invoke({"input_data": extracted_html, "product_id": product_id})

        # 결과 출력
        # print(result.content)

        # 파일에 저장
        import json
        import os

        new_data_text = result.content  # LLM 응답 JSON 문자열

        try:
            new_data = json.loads(new_data_text)
        except json.JSONDecodeError as error:
            print("JSON 파싱 에러:", error)
            print("new_data_text : ", new_data_text)
            new_data = None
            continue

        if new_data is None:
            print("유효한 JSON 데이터가 아니므로 종료합니다.")
        else:
            filename = '../data/raw_docs/product_info.json'

        # 기존 데이터 불러오기 (파일 없으면 빈 리스트로 시작)
        if os.path.exists(filename):
            with open(filename, 'r', encoding='utf-8') as f:
                try:
                    existing_data = json.load(f)
                except json.JSONDecodeError:
                    existing_data = []
        else:
            existing_data = []

        # existing_data가 리스트인지 확인. 아니면 리스트로 감싸기
        if not isinstance(existing_data, list):
            existing_data = [existing_data]

        # 같은 name 있는지 검사
        exists = any(item.get('name') == new_data.get('name') for item in existing_data)

        if exists:
            print(f"'{new_data.get('name')}' 상품이 이미 존재합니다. 추가하지 않습니다.")
        else:
            existing_data.append(new_data)
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(existing_data, f, ensure_ascii=False, indent=2)
            print(f"'{new_data.get('name')}' 상품 정보를 추가 저장했습니다.")

    except Exception as error:
        print(f"[{idx}] 에러 발생:", error)
        continue
    finally:
        s = time()
        print(f"{s-e} 초 걸렸습니다")


In [ ]:
import json
with open("../data/raw_docs/product_info.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    print(f"총 {len(data)} 개의 상품 정보가 저장되었습니다.")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings 등
from langchain.docstore.document import Document   # 문서 구조화할 때 사용

vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(),
    persist_directory="../data/vectordb_chroma",  # 벡터 저장 디렉토리
)

results = vectorstore.get()  # 전체 조회
print(results["documents"])  # 문서 내용 출력

# import shutil

# shutil.rmtree("../data/vectordb_chroma")  # 경로는 본인의 persist_directory에 맞게 수정
